In [13]:
import sys
from pathlib import Path

# Añade src al path para importar los módulos
sys.path.append(str(Path().resolve().parent / 'src'))

In [14]:
%reload_ext autoreload
%autoreload 2

In [15]:
from src.data_utils import load_raw_data
import pandas as pd

# Cargar los datos
df_raw = load_raw_data(
    fecha_inicio="2023-01-02", 
    fecha_fin="2025-06-30", 
    descargar_bq=False)

df_raw.describe()



Cargando datos desde: C:\Workspace\mlops_fleca_project\data\raw\raw_data_bq_forecasting_20250630.parquet
Total de fechas faltantes: 4
Fechas faltantes: ['2023-12-25T00:00:00.000000000' '2024-01-01T00:00:00.000000000'
 '2024-12-25T00:00:00.000000000' '2025-01-01T00:00:00.000000000']


,fecha,cantidad,base_imponible,tipo_IVA,total,is_summer_peak,is_easter
count,337353,337353.000000,337353.000000,335106.000000,337353.000000,337353.000000,337353.0
mean,2024-03-10 00:54:30.032281344,1.209303,2.010886,9.084558,2.199180,0.208473,0.0
min,2023-01-02 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,2023-07-28 00:00:00,1.000000,1.350000,10.000000,1.450000,0.000000,0.0
50%,2024-03-09 00:00:00,1.000000,1.640000,10.000000,1.800000,0.000000,0.0
75%,2024-09-25 00:00:00,1.000000,2.320000,10.000000,2.550000,0.000000,0.0
max,2025-06-30 00:00:00,93.000000,287.270000,10.000000,316.000000,1.000000,0.0
std,NaN,0.733263,1.692278,2.793367,1.862156,0.406217,0.0


In [16]:
from src.data_utils import transformar_a_series_temporales

df_ts = transformar_a_series_temporales(df_raw)

    


In [17]:
df_ts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   year            126 non-null    UInt32        
 1   week            126 non-null    UInt32        
 2   familia         126 non-null    object        
 3   base_imponible  126 non-null    float64       
 4   is_summer_peak  126 non-null    int32         
 5   is_easter       126 non-null    int64         
 6   dias_semana     126 non-null    int64         
 7   week_start      126 non-null    datetime64[ns]
dtypes: UInt32(2), datetime64[ns](1), float64(1), int32(1), int64(2), object(1)
memory usage: 6.8+ KB


In [20]:
import os
from dotenv import load_dotenv
from paths import ROOT_DIR
import hopsworks

load_dotenv(os.path.join(ROOT_DIR, '.env'))

HOPSWORKS_PROJECT_NAME = 'fleca_mlops'
HOPSWORKS_API_KEY = os.getenv('HOPSWORKS_API_KEY')  

project = hopsworks.login(
    api_key_value=HOPSWORKS_API_KEY, 
    project=HOPSWORKS_PROJECT_NAME)

feature_store = project.get_feature_store()

2025-08-07 20:39:00,920 INFO: Initializing external client
2025-08-07 20:39:00,921 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-08-07 20:39:12,210 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242272
